# Revised case normalization for Hirslanden Zurich 2018

Notes:
- There is 'n.ü.' in the duration_of_stay, at the moment repöace them with nan



In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os
import numpy as np
from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
        'Hirslanden Zurich',
         '2018',
        ['Änderungen_Hirslanden_2018'])

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Zurich', year='2018', sheets=['Änderungen_Hirslanden_2018'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

try:
    df_revised_case_d2d = normalize(file_info, 0)
except ValueError:
    print(str(sys.exc_info()))

Read 120 cases for Hirslanden Zurich 2018
4/120 rows were deleted because contained NaNs
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-17 15:21:53.779 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 31: discarded ICDs after validation {''}
2022-10-17 15:21:53.784 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-17 15:21:53.785 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 3: discarded ICDs after validation {''}
2022-10-17 15:21:53.785 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 5: discarded ICDs after validation {''}
2022-10-17 15:21:53.786 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 6: discarded ICDs after validation {''}
2022-10-17 15:21:53.787 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 7: discarded ICDs after validation {''}
2022-10-17 15:21:53.787 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 12: discarded ICDs after validation {''}
2022-10-17 15:21:53.788 | DEBUG    | src.utils.dataframe_utils:_validate_i

In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,0041622923,D57E1A8A84BA5715,M,76,19,3,G17Z,C20,C20,M200,[E440],[E441],[],[]
1,0041628878,1EAE82BF47B46A42,M,84,5,3,R60E,C9200,C9200,M100,[B909],[],[],[]
2,0041634840,69B604C8EDC65FDA,M,66,17,3,X06B,T818,T818,M100,"[E1173, T842, I5013]","[E1121, E1151, E1141]",[],[]
3,0041643502,D3F33C4023F17E75,W,72,14,3,I08C,M161,M161,M200,[D62],[],[],[]
4,0041657438,B26B73105A4B1FE3,M,66,8,3,H01B,K835,K835,M200,"[D1803, I420]","[D134, Z867]",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0044091599,FF1CA55986D937DB,M,75,7,3,G60B,C180,C180,M100,[E43],[],[],[]
116,0044093471,F44CB526CF8A238E,W,55,6,3,F75D,T827,T827,M100,[D592],[D648],[],[]
117,0044094461,9FDAF84F7F440BFA,W,82,7,3,L63D,N10,N10,M100,"[U804, B966]",[],[],[]
118,0044098568,CDD9C616F85275E8,W,64,12,3,F06B,I351,I351,M200,"[E871, I5001, I652]",[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['0041622923', '0041628878', '0041634840', '0041643502',
       '0041657438', '0041663091', '0041667883', '0041681234',
       '0041599301', '0041607224', '0041609140', '0041640963',
       '0041644444', '0041645715', '0041645764', '0041654006',
       '0041660151', '0041661822', '0041677400', '0041625950',
       '0041628307', '0041629305', '0041636297', '0041689614',
       '0041689771', '0041697972', '0041703740', '0041709733',
       '0041712923', '0041704484', '0041706998', '0041708001',
       '0041710019', '0041712227', '0041712478', '0041715111',
       '0041727258', '0041729677', '0041733974', '0044000509',
       '0044004182', '0044006689', '0044006827', '0041733872',
       '0044003361', '0044007343', '0044008464', '0044009706',
       '0044010330', '0044011741', '0044011898', '0044016534',
       '0044016444', '0044016672', '0044029391', '0044032804',
       '0044042078', '0044042175', '0044045394', '0044048088',
       '0044048666', '0044049149', '0044049518', '00440

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

In [7]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,2.228,156769,1,0041622923,D57E1A8A84BA5715,76,0,M,19,4,0,0,0,2018-01-17,01,2018-02-05,00,G17Z,G17,3
0,1.496,157016,1,0041628878,1EAE82BF47B46A42,84,0,M,5,3,0,0,0,2018-01-03,01,2018-01-08,00,R60E,R60,3
0,1.062,157207,1,0041634840,69B604C8EDC65FDA,66,0,M,17,3,0,0,0,2018-01-06,01,2018-01-23,00,X06B,X06,3
0,2.320,157547,1,0041643502,D3F33C4023F17E75,72,0,W,14,4,0,0,0,2018-01-22,01,2018-02-05,00,I08C,I08,3
0,3.359,158153,1,0041657438,B26B73105A4B1FE3,66,0,M,8,4,0,0,0,2018-02-01,01,2018-02-09,00,H01B,H01,3


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,2.228,156769,1,0041622923,D57E1A8A84BA5715,76,0,M,19,4,...,0,0,2018-01-17,01,2018-02-05,00,G17Z,G17,3,41622923
0,1.496,157016,1,0041628878,1EAE82BF47B46A42,84,0,M,5,3,...,0,0,2018-01-03,01,2018-01-08,00,R60E,R60,3,41628878
0,1.062,157207,1,0041634840,69B604C8EDC65FDA,66,0,M,17,3,...,0,0,2018-01-06,01,2018-01-23,00,X06B,X06,3,41634840
0,2.320,157547,1,0041643502,D3F33C4023F17E75,72,0,W,14,4,...,0,0,2018-01-22,01,2018-02-05,00,I08C,I08,3,41643502
0,3.359,158153,1,0041657438,B26B73105A4B1FE3,66,0,M,8,4,...,0,0,2018-02-01,01,2018-02-09,00,H01B,H01,3,41657438


In [10]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()


,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped
0,0041622923,D57E1A8A84BA5715,M,76,19,3,G17Z,C20,C20,M200,[E440],[E441],[],[],41622923
1,0041628878,1EAE82BF47B46A42,M,84,5,3,R60E,C9200,C9200,M100,[B909],[],[],[],41628878
2,0041634840,69B604C8EDC65FDA,M,66,17,3,X06B,T818,T818,M100,"[E1173, T842, I5013]","[E1121, E1151, E1141]",[],[],41634840
3,0041643502,D3F33C4023F17E75,W,72,14,3,I08C,M161,M161,M200,[D62],[],[],[],41643502
4,0041657438,B26B73105A4B1FE3,M,66,8,3,H01B,K835,K835,M200,"[D1803, I420]","[D134, Z867]",[],[],41657438


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [11]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(116, 28)


/tmp/ipykernel_14540/3233428173.py:4: FutureWarning: Passing 'suffixes' as a <class 'NoneType'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)


,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped,gender,age_years,...,age_days,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,adrg
0,C20,C20,M200,[E440],[E441],[],[],41622923,M,76,...,0,4,0,0,0,2018-01-17,01,2018-02-05,00,G17
1,C9200,C9200,M100,[B909],[],[],[],41628878,M,84,...,0,3,0,0,0,2018-01-03,01,2018-01-08,00,R60
2,T818,T818,M100,"[E1173, T842, I5013]","[E1121, E1151, E1141]",[],[],41634840,M,66,...,0,3,0,0,0,2018-01-06,01,2018-01-23,00,X06
3,M161,M161,M200,[D62],[],[],[],41643502,W,72,...,0,4,0,0,0,2018-01-22,01,2018-02-05,00,I08
4,K835,K835,M200,"[D1803, I420]","[D134, Z867]",[],[],41657438,M,66,...,0,4,0,0,0,2018-02-01,01,2018-02-09,00,H01


In [12]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_SELECT]
df_revised_case_normalized.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,0041622923,D57E1A8A84BA5715,M,76,19,3,G17Z,C20,C20,M200,[E440],[E441],[],[]
1,0041628878,1EAE82BF47B46A42,M,84,5,3,R60E,C9200,C9200,M100,[B909],[],[],[]
2,0041634840,69B604C8EDC65FDA,M,66,17,3,X06B,T818,T818,M100,"[E1173, T842, I5013]","[E1121, E1151, E1141]",[],[]
3,0041643502,D3F33C4023F17E75,W,72,14,3,I08C,M161,M161,M200,[D62],[],[],[]
4,0041657438,B26B73105A4B1FE3,M,66,8,3,H01B,K835,K835,M200,"[D1803, I420]","[D134, Z867]",[],[]


# Export the normalized revised cases with the neccessary columns to a csv file


In [13]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [14]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 116 out of 116 revised cases from DtoD that can be matched with the bfs database data for Hirslanden Zurich 2018


In [15]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/Hirslanden_Zurich_2018.csv
